# POS TAGGING USING MODIEFIED VITERBI

#### SREENATH S

## OBJECTIVE:

Modify the Viterbi algorithm to solve the problem of unknown words using at least two techniques. Though there could be multiple ways to solve this problem, you may use the following hints:

Which tag class do you think most unknown words belong to? Can you identify rules (e.g. based on morphological cues) that can be used to tag unknown words? You may define separate python functions to exploit these rules so that they work in tandem with the original Viterbi algorithm.


Why does the Viterbi algorithm choose a random tag on encountering an unknown word? Can you modify the Viterbi algorithm so that it considers only one of the transition or emission probabilities for unknown words?

### DATA PREPERATION

Importing required libraries

In [1]:
import nltk
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from collections import Counter

Read the treebank tagged sentences

In [2]:
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# first few tagged sentences
print(nltk_data[:5])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# Splitting into train and test
train_set, test_set = train_test_split(nltk_data,train_size = 0.95, test_size=0.05, random_state = 42)

print("Training set length: ", len(train_set))
print("\nTest set length: ", len(test_set))
print("\nTraining Set Data at a glance: \n\n", train_set[:5])

Training set length:  3718

Test set length:  196

Training Set Data at a glance: 

 [[('Bank', 'NOUN'), ('of', 'ADP'), ('New', 'NOUN'), ('England', 'NOUN'), ("'s", 'PRT'), ('shares', 'NOUN'), ('are', 'VERB'), ('traded', 'VERB'), ('*-1', 'X'), ('on', 'ADP'), ('the', 'DET'), ('New', 'NOUN'), ('York', 'NOUN'), ('Stock', 'NOUN'), ('Exchange', 'NOUN'), ('.', '.')], [('$', '.'), ('130', 'NUM'), ('million', 'NUM'), ('*U*', 'X'), ('of', 'ADP'), ('general', 'ADJ'), ('obligation', 'NOUN'), ('distributable', 'ADJ'), ('state', 'NOUN'), ('aid', 'NOUN'), ('bonds', 'NOUN'), ('due', 'ADJ'), ('1991-2000', 'NUM'), ('and', 'CONJ'), ('2009', 'NUM'), (',', '.'), ('tentatively', 'ADV'), ('priced', 'VERB'), ('*', 'X'), ('by', 'ADP'), ('a', 'DET'), ('Chemical', 'NOUN'), ('Securities', 'NOUN'), ('Inc.', 'NOUN'), ('group', 'NOUN'), ('*', 'X'), ('to', 'PRT'), ('yield', 'VERB'), ('from', 'ADP'), ('6.20', 'NUM'), ('%', 'NOUN'), ('in', 'ADP'), ('1991', 'NUM'), ('to', 'PRT'), ('7.272', 'NUM'), ('%', 'NOUN'), ('in',

A quick observation from above tagged words is that, whenever '*' appers with some alpha-numeric characters it is tagged as 'X'.
Also when "xyz-abc"occurs it is tagged as ADJ

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
print("Number of tagged words in Training Set: ", len(train_tagged_words))

Number of tagged words in Training Set:  95589


In [6]:
# Getting list of tagged words
test_tagged_words = [tup for sent in test_set for tup in sent]
print("Number of tagged words in Test Set: ", len(test_tagged_words))

Number of tagged words in Test Set:  5087


In [7]:
# tokens 
train_words = [pair[0] for pair in train_tagged_words]
print("Train words (first 10): ", train_words[:10])

Train words (first 10):  ['Bank', 'of', 'New', 'England', "'s", 'shares', 'are', 'traded', '*-1', 'on']


In [8]:
train_unique_words = set(train_words)
vocab_size = len(train_unique_words)
print("Vocabulary size: ", vocab_size)

Vocabulary size:  12109


In [9]:
# tokens 
train_pos_tags = [pair[1] for pair in train_tagged_words]
print("Train tags (first 10): ", train_pos_tags[:10])

Train tags (first 10):  ['NOUN', 'ADP', 'NOUN', 'NOUN', 'PRT', 'NOUN', 'VERB', 'VERB', 'X', 'ADP']


In [10]:
train_unique_tags = set(train_pos_tags)
train_unique_tags = sorted(train_unique_tags)
num_unique_tags = len(train_unique_tags)
print("Number of unique tags in train set: ", num_unique_tags)

Number of unique tags in train set:  12


In [11]:
tag_counts = Counter(train_pos_tags)
tag_counts

Counter({'NOUN': 27423,
         'ADP': 9345,
         'PRT': 3059,
         'VERB': 12885,
         'X': 6276,
         'DET': 8284,
         '.': 11118,
         'NUM': 3363,
         'ADJ': 6077,
         'CONJ': 2131,
         'ADV': 3010,
         'PRON': 2618})

## BUILD THE VANILLA VITERBI BASED ON POS TAGGER

Utility function to calculate the emission probability. Emission probability = (number of times a word with given tag present in the corpora/ number of time the given tag is present in the corpora)

Below function returns a tuple containing:

number of times a word with given tag present in the corpora
number of time the given tag is present in the corpora

In [12]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

Utility function to calculate the transition probability. Transition probability = (number of times tag T1 followed by tag T2/ number of times tag T1 present in the corpora)

Below function returns a tuple containing:

number of times tag T1 followed by tag T2
number of times tag T1 present in the corpora

In [13]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [14]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((num_unique_tags, num_unique_tags), dtype='float32')
for i, t1 in enumerate(list(train_unique_tags)):
    for j, t2 in enumerate(list(train_unique_tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(train_unique_tags), index=list(train_unique_tags))
tags_df

,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
.,0.093812,0.044972,0.091114,0.052078,0.057924,0.173502,0.223152,0.080500,0.065389,0.002339,0.088505,0.026623
ADJ,0.065328,0.066645,0.078986,0.004608,0.016949,0.005101,0.696725,0.020405,0.000658,0.010861,0.012342,0.021392
ADP,0.039486,0.105297,0.017228,0.013162,0.000856,0.326378,0.321776,0.062921,0.069128,0.001498,0.008240,0.034029
ADV,0.136877,0.130233,0.119601,0.081063,0.006312,0.067110,0.030897,0.030565,0.015615,0.013621,0.344518,0.023588
CONJ,0.034256,0.116847,0.054435,0.055842,0.000469,0.121539,0.349132,0.042234,0.058658,0.004693,0.153918,0.007977
DET,0.017986,0.204973,0.009054,0.012313,0.000483,0.005311,0.640029,0.021970,0.003742,0.000241,0.038387,0.045509
NOUN,0.239179,0.012289,0.176275,0.016884,0.041936,0.013310,0.264632,0.009627,0.004923,0.043832,0.147978,0.029136
NUM,0.118347,0.034196,0.034790,0.002974,0.013381,0.003271,0.355338,0.184062,0.001487,0.027951,0.017544,0.206661
PRON,0.041253,0.074866,0.022918,0.033995,0.004966,0.009549,0.211230,0.007257,0.008021,0.012223,0.480901,0.092819
PRT,0.041517,0.086303,0.021576,0.010134,0.002288,0.100360,0.242563,0.058516,0.018960,0.001635,0.402746,0.013403


### VANILLA VITERBI:

Used the Vanilla Viterbi implementation given as part of "Session2 - Syntactic Processing" with some modification to reduce the computation complexity. In the implementation given in "Session2 - Syntactic Processing" there are two times the emission probability gets calculated. Which is unncessary, hence in this implementation its computed only once.

In [16]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    # init state array to empty - state array holds the POS tag for words
    # T - Is the list of unique tags
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    T = sorted(T)
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            proba_pair = word_given_tag(words[key], tag)
            emission_p = proba_pair[0]/proba_pair[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax, 0)] 
        state.append(state_max)
    return list(zip(words, state))

## SOLVE THE PROBLEM OF UNKNOWN WORDS:

### MODIFICATION 1: ONLY CONSIDERING TRANSITION PROBABILITY FOR UNKNOWN WORDS

For unknown words, ie if the word is not present in training vocabulary, then its emission probability will be 0, as the word count will be zero.

Emission probability = (number of times a give word with given tag present in the corpora/ number of time the given tag is present in the corpora)

Hence as per the hints given, we can only consider the transition probability for POS tagging for unknown words. Vanilla Viterbi method will be modified to check whether the word exists in training corpora, if it exists then only calculate transition and emission probability. Otherwise calculate only transition probability (This way avoid one morecomputation heavy step)

In [17]:
# Viterbi Heuristic
def Viterbi_with_transition_proba(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    T = sorted(T)
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            # check whether the word exists in training corpora, 
            # if it exists then only calculate transition and emission probability. 
            # Otherwise calculate only transition probability
            if word in train_words:
                proba_pair = word_given_tag(words[key], tag)
                emission_p = proba_pair[0]/proba_pair[1]
                state_probability = emission_p * transition_p
            else:
                state_probability = transition_p
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax, 0)] 
        state.append(state_max)
    return list(zip(words, state))

### MODIFICATION 2: VITERBI ALGORITHM WITH REGEX TAGGER BACKOFF

As we have already noted there are some visible pattern in training set:
    
    Anuything with * is tagged as X
    Starting with a numeric 1, 11.0, 12,345, 7-8 all are tagged as NUM
    Any two words joined with - is tagged as ADJ
    All puctuations with .
    
Along with this we will apply general english ruleset as regex pattern. Any word which does not match with any of the rule will be tagged as UNK. Those words will be tagged based on transition probability.

English generic ruleset courtesy: 
https://www.nltk.org/book/ch05.html

https://www.kaggle.com/saxinou/nlp-02-categorizing-and-tagging-words

In [18]:
patterns = [
    (r'^\*+[*\w-]*', 'X'),                                          # *[A-z] - X
    (r'^[0-9.,-]+[A-z]*$', 'NUM'),                                  # 9.00, 9,000, 9-8
    (r'^([a-z]+(-[\w]+)+)+$', 'ADJ'),                               # abc - xyz - ADJ
    (r'^[A-Z]+[a-z]*.*$', 'NOUN'),                                  # Starting with caps, All Caps etc
    (r'^[A-z]+s$', 'NOUN'),
    (r'^[A-z]+ing$', 'VERB'),                                       # gerunds
    (r'^[A-z]+ed$', 'VERB'),                                        # simple past
    (r'^[A-z]+es$', 'VERB'), 
    (r'^\'s$', 'VERB'),   
    (r'^[A-z]+\'s$', 'NOUN'),               
    (r'(The|the|A|a|An|an)$', 'DET'),                               # Determinants 
    (r'^[A-z]+able$', 'ADJ'),                                       # adjectives 
    (r'^[A-z]+ness$', 'NOUN'),                                      # nouns formed from adjectives
    (r'^[A-z]+ly$', 'ADV'),                           
    (r'(He|he|She|she|It|it|I|me|Me|You|you)$', 'PRON'), 
    (r'(His|his|Her|her|Its|its)$', 'PRON'),      
    (r'(My|my|Your|your|Yours|yours)$', 'PRON'),    
    (r'(Myself|myself|Yourself|yourself|herself|himself)$', 'PRON'), 
    (r'(themselves|ourselves)$', 'PRON'),                           # possesive
    (r'(on|On|in|In|at|At|since|Since)$', 'ADP'),                   # time prepopsitions
    (r'(for|For|ago|Ago|before|Before)$', 'ADP'),                   # time prepopsitions
    (r'(till|Till|until|Until)$', 'ADP'),                           # time prepopsitions
    (r'(by|By|beside|Beside)$', 'ADP'),                             # space prepopsitions
    (r'(under|Under|below|Below)$', 'ADP'),                         # space prepopsitions
    (r'(up|Up|down|Down)$', 'ADP'),                                 # space prepopsitions
    (r'(over|Over|above|Above)$', 'ADP'),                           # space prepopsitions
    (r'(across|Across|through|Through)$', 'ADP'),                   # space prepopsitions
    (r'(into|Into|towards|Towards)$', 'ADP'),                       # space prepopsitions
    (r'(onto|Onto|from|From)$', 'ADP'),                             # space prepopsitions    
    (r'^\.+$','.'), (r'^\,+$',','), (r'^\?+$','.'),                 # punctuations
    (r'^\'+$','.'), (r'^\"+$',','), (r'^\`+$','.'),                 # punctuations
    (r'.*', 'UNK')                                                  # UNKNOWN (default)
    ]


Regex tagger based on the above ruleset, this will serve as the backoff for modified viterbi

In [19]:
regex_tagger = nltk.RegexpTagger(patterns)

If the emission probability is 0, ie word is unknown, then the regex based tagger will be used to tag the POS. But if regex based tagger fails to tag we will use the transition probability alone for tagging.

In [20]:
# Viterbi Heuristic
def Viterbi_with_regex_backoff(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    T = sorted(T)
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            if word in train_words:
                proba_pair = word_given_tag(words[key], tag)
                emission_p = proba_pair[0]/proba_pair[1]
                state_probability = emission_p * transition_p
            else:
                state_probability = transition_p
            p.append(state_probability)

        if word in train_words:
            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax, 0)] 
        else:
            state_max = regex_tagger.tag([word])[0][1]
            if state_max == "UNK":
                pmax = max(p)
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

## EVALUATING TAGGING ACCURACY

**NOTE: THE ENTIRE BELOW SECTION MAY TAKE 1HR to 1.5HR MINIMUM**

Training set on all 3 different algos, ie Vanilla viterbi, Viterbi with Modification 1 and 2.

Let us get the tagged tuple, as well as words from the test set for feeding on to the viterbi models.

In [21]:
# list of tagged words from test set (5% sentence)
test_run_base = [tup for sent in test_set for tup in sent]
# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
len(test_tagged_words)

5087

### 1. VANILLA VITERBI ON TEST SET (CREATED BY TRAIN TEST SPLIT):

In [22]:
# tagging the test sentences
start = time.time()
tagged_seq_vanilla_viterbi = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  798.2135455608368


In [23]:
test_check_vanilla = [i for i, j in zip(tagged_seq_vanilla_viterbi, test_run_base) if i == j] 
print("Number of words correctly tagged: ", len(test_check_vanilla))
print("Total Number of words: ", len(tagged_seq_vanilla_viterbi))
accuracy = len(test_check_vanilla)/len(tagged_seq_vanilla_viterbi)
print("Accuracy: ", accuracy)

Number of words correctly tagged:  4649
Total Number of words:  5087
Accuracy:  0.9138981718104974


**Below are incorrectly tagged words by Vanilla Viterbi**

In [24]:
incorrect_tagged_cases = [j for i, j in enumerate(zip(tagged_seq_vanilla_viterbi, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[(('trade', 'VERB'), ('trade', 'NOUN')),
 (('Overseas', '.'), ('Overseas', 'NOUN')),
 (('Private', 'ADJ'), ('Private', 'NOUN')),
 (('pre-1917', '.'), ('pre-1917', 'ADJ')),
 (('Unemployment', '.'), ('Unemployment', 'NOUN')),
 (('purchasing', 'NOUN'), ('purchasing', 'VERB')),
 (('paycheck', '.'), ('paycheck', 'NOUN')),
 (('reasonably', '.'), ('reasonably', 'ADV')),
 (('though', 'ADP'), ('though', 'ADV')),
 (('close', 'NOUN'), ('close', 'ADJ')),
 (('more', 'ADV'), ('more', 'ADJ')),
 (('Funded', '.'), ('Funded', 'VERB')),
 (('Tokio', '.'), ('Tokio', 'NOUN')),
 (('protocols', '.'), ('protocols', 'NOUN')),
 (('preventative', '.'), ('preventative', 'ADJ')),
 (('existed', '.'), ('existed', 'VERB')),
 (('20.5', '.'), ('20.5', 'NUM')),
 (('polarized', '.'), ('polarized', 'VERB')),
 (('so', 'ADV'), ('so', 'ADP')),
 (('exists', '.'), ('exists', 'VERB')),
 (('middle', 'NOUN'), ('middle', 'ADJ')),
 (('Though', '.'), ('Though', 'ADP')),
 (('acquirers', '.'), ('acquirers', 'NOUN')),
 (('noticed', '.')

### 2. MODIFIED VITERBI WITH TRANSITION PROBABILITY ON TEST SET (CREATED BY TRAIN TEST SPLIT):

In [25]:
# tagging the test sentences
start = time.time()
tagged_seq_mod_viterbi_1 = Viterbi_with_transition_proba(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  773.376373052597


In [26]:
test_check_mod_viterbi_1 = [i for i, j in zip(tagged_seq_mod_viterbi_1, test_run_base) if i == j] 
print("Number of words correctly tagged: ", len(test_check_mod_viterbi_1))
print("Total Number of words: ", len(tagged_seq_mod_viterbi_1 ))
accuracy = len(test_check_mod_viterbi_1)/len(tagged_seq_mod_viterbi_1)
print("Accuracy: ", accuracy)

Number of words correctly tagged:  4776
Total Number of words:  5087
Accuracy:  0.9388637703951248


**Below are incorrectly tagged words by Viterbi with only transition proba for unknown words**

In [27]:
incorrect_tagged_cases_viterbi_transition_proba = [j for i, j in enumerate(zip(tagged_seq_mod_viterbi_1, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_viterbi_transition_proba

[(('trade', 'VERB'), ('trade', 'NOUN')),
 (('Private', 'ADJ'), ('Private', 'NOUN')),
 (('pre-1917', 'X'), ('pre-1917', 'ADJ')),
 (('purchasing', 'NOUN'), ('purchasing', 'VERB')),
 (('reasonably', 'NOUN'), ('reasonably', 'ADV')),
 (('though', 'ADP'), ('though', 'ADV')),
 (('close', 'NOUN'), ('close', 'ADJ')),
 (('more', 'ADV'), ('more', 'ADJ')),
 (('Tokio', 'DET'), ('Tokio', 'NOUN')),
 (('preventative', 'DET'), ('preventative', 'ADJ')),
 (('20.5', 'NOUN'), ('20.5', 'NUM')),
 (('polarized', 'X'), ('polarized', 'VERB')),
 (('so', 'ADV'), ('so', 'ADP')),
 (('middle', 'NOUN'), ('middle', 'ADJ')),
 (('Though', 'NOUN'), ('Though', 'ADP')),
 (('154.2', 'X'), ('154.2', 'NUM')),
 (('chalk', 'DET'), ('chalk', 'NOUN')),
 (('first', 'ADJ'), ('first', 'ADV')),
 (('touched', 'NOUN'), ('touched', 'VERB')),
 (('cross-border', 'DET'), ('cross-border', 'ADJ')),
 (('emigres', 'DET'), ('emigres', 'NOUN')),
 (('*T*-133', 'NOUN'), ('*T*-133', 'X')),
 (('checking', 'NOUN'), ('checking', 'VERB')),
 (('safe', '

### 3. MODIFIED VITERBI WITH TREGEX TAGGER BACKOFF ON TEST SET (CREATED BY TRAIN TEST SPLIT):

In [28]:
# tagging the test sentences
start = time.time()
tagged_seq_mod_viterbi_2 = Viterbi_with_regex_backoff(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  793.2432219982147


In [29]:
test_check_mod_viterbi_2 = [i for i, j in zip(tagged_seq_mod_viterbi_2, test_run_base) if i == j] 
print("Number of words correctly tagged: ", len(test_check_mod_viterbi_2))
print("Total Number of words: ", len(tagged_seq_mod_viterbi_2 ))
accuracy = len(test_check_mod_viterbi_2)/len(tagged_seq_mod_viterbi_2)
print("Accuracy: ", accuracy)

Number of words correctly tagged:  4897
Total Number of words:  5087
Accuracy:  0.9626498918812659


**Below are incorrectly tagged words by Viterbi with regex based POS tagger for unknown words**

In [30]:
incorrect_tagged_cases_regex_boff = [j for i, j in enumerate(zip(tagged_seq_mod_viterbi_2, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_regex_boff

[(('trade', 'VERB'), ('trade', 'NOUN')),
 (('Private', 'ADJ'), ('Private', 'NOUN')),
 (('purchasing', 'NOUN'), ('purchasing', 'VERB')),
 (('though', 'ADP'), ('though', 'ADV')),
 (('close', 'NOUN'), ('close', 'ADJ')),
 (('more', 'ADV'), ('more', 'ADJ')),
 (('Funded', 'NOUN'), ('Funded', 'VERB')),
 (('preventative', 'DET'), ('preventative', 'ADJ')),
 (('so', 'ADV'), ('so', 'ADP')),
 (('exists', 'NOUN'), ('exists', 'VERB')),
 (('middle', 'NOUN'), ('middle', 'ADJ')),
 (('Though', 'NOUN'), ('Though', 'ADP')),
 (('chalk', 'DET'), ('chalk', 'NOUN')),
 (('first', 'ADJ'), ('first', 'ADV')),
 (('slate', 'X'), ('slate', 'NOUN')),
 (('checking', 'NOUN'), ('checking', 'VERB')),
 (('safe', 'NOUN'), ('safe', 'ADJ')),
 (('plus', 'CONJ'), ('plus', 'ADV')),
 (('homeless', 'ADJ'), ('homeless', 'NOUN')),
 (('up', 'ADV'), ('up', 'PRT')),
 (('American-style', 'NOUN'), ('American-style', 'ADJ')),
 (('most', 'ADJ'), ('most', 'ADV')),
 (('that', 'ADP'), ('that', 'DET')),
 (('as', 'ADP'), ('as', 'ADV')),
 (('te

## COMPARE TAGGING ACCURACIES OF THE MODIFICATIONS WITH THE VANILLA VITERBI ALGORITHM:

**FOLLOWING ARE THE ACCURACIES OF VITERBI ALGORITHMS ON TEST SET:**
    
**1. VANILLA VITERBI: 91.39%**

**2. VITERBI WITH TRANSITION PROBABILITY FOR UNKNOWN WORDS: 93.89%**

**3. VITERBI WITH REGEX POS TAGGING: 96.26%**

### EVALUATION ON TEST SENTENCES AS GIVEN PART OF "Test_sentences.txt"

Read the data from file and prepare the input data for viterbi algorithm:

In [32]:
with open("Test_sentences.txt") as file: 
    Lines = file.readlines() 
    Lines = [line.strip() for line in Lines] 

print(Lines)

['Android is a mobile operating system developed by Google.', 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.', "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.", 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.', 'Before entering politics, Donald Trump was a domineering businessman and a television personality.', 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.', 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.', 'Show me the cheapest round trips from Dallas to Atlanta', 'I would like to see flights from Denver to Philadelphia.', 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.', 'NASA invited social media users to experience the laun

Tokenize the evaluation sentences

In [33]:
evaluation_sentences = [word_tokenize(line) for line in Lines]
evaluation_sentences = [sent for sent in evaluation_sentences]
print("evaluation_sentencese: ", evaluation_sentences)

evaluation_sentencese:  [['Android', 'is', 'a', 'mobile', 'operating', 'system', 'developed', 'by', 'Google', '.'], ['Android', 'has', 'been', 'the', 'best-selling', 'OS', 'worldwide', 'on', 'smartphones', 'since', '2011', 'and', 'on', 'tablets', 'since', '2013', '.'], ['Google', 'and', 'Twitter', 'made', 'a', 'deal', 'in', '2015', 'that', 'gave', 'Google', 'access', 'to', 'Twitter', "'s", 'firehose', '.'], ['Twitter', 'is', 'an', 'online', 'news', 'and', 'social', 'networking', 'service', 'on', 'which', 'users', 'post', 'and', 'interact', 'with', 'messages', 'known', 'as', 'tweets', '.'], ['Before', 'entering', 'politics', ',', 'Donald', 'Trump', 'was', 'a', 'domineering', 'businessman', 'and', 'a', 'television', 'personality', '.'], ['The', '2018', 'FIFA', 'World', 'Cup', 'is', 'the', '21st', 'FIFA', 'World', 'Cup', ',', 'an', 'international', 'football', 'tournament', 'contested', 'once', 'every', 'four', 'years', '.'], ['This', 'is', 'the', 'first', 'World', 'Cup', 'to', 'be', 'hel

#### Tag the data using nltk tagger to compare accuracy.

Let us tag the evaluation sentences using the nltk pos_tag_sents. This will serve as a tagged baseline for us to compare the accuracy of our model

In [34]:
evaluation_sentences_run = nltk.pos_tag_sents(evaluation_sentences, tagset = 'universal')
print(evaluation_sentences_run)

[[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')], [('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.')], [('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'DET'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.')], [('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'ADJ'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DE

We can see Atlanta incorrectly tagged as VERB. Lets correct it as NOUN

In [35]:
evaluation_sentences_run_base = [tup for sent in evaluation_sentences_run for tup in sent]
for i, tup in enumerate(evaluation_sentences_run_base):
    if tup[0] == 'Atlanta':
        evaluation_sentences_run_base[i] = ('Atlanta', 'NOUN')
evaluation_sentences_words = [tup[0] for sent in evaluation_sentences_run for tup in sent]

### 1. EVALUATION ON VANILLA VITERBI:

In [36]:
# tagging the test sentences
start = time.time()
eval_tagged_seq_vanilla_viterbi = Viterbi(evaluation_sentences_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  59.968430280685425


In [37]:
sample_check_vanilla = [i for i, j in zip(eval_tagged_seq_vanilla_viterbi, evaluation_sentences_run_base) if i == j] 
print("Number of words correctly tagged: ", len(sample_check_vanilla))
print("Total Number of words: ", len(eval_tagged_seq_vanilla_viterbi))
accuracy = len(sample_check_vanilla)/len(eval_tagged_seq_vanilla_viterbi)
print("Accuracy: ", accuracy)

Number of words correctly tagged:  138
Total Number of words:  181
Accuracy:  0.7624309392265194


**Incorrectly tagged words by vanilla viterbi**

In [38]:
incorrect_tagged_cases_vanilla_viterbi = [j for i, j in enumerate(zip(eval_tagged_seq_vanilla_viterbi, evaluation_sentences_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_vanilla_viterbi

[(('Android', '.'), ('Android', 'NOUN')),
 (('Google', '.'), ('Google', 'NOUN')),
 (('Android', '.'), ('Android', 'NOUN')),
 (('OS', '.'), ('OS', 'NOUN')),
 (('worldwide', '.'), ('worldwide', 'NOUN')),
 (('smartphones', '.'), ('smartphones', 'NOUN')),
 (('2011', '.'), ('2011', 'NUM')),
 (('2013', '.'), ('2013', 'NUM')),
 (('Google', '.'), ('Google', 'NOUN')),
 (('Twitter', '.'), ('Twitter', 'NOUN')),
 (('2015', '.'), ('2015', 'NUM')),
 (('Google', '.'), ('Google', 'NOUN')),
 (('Twitter', '.'), ('Twitter', 'NOUN')),
 (("'s", 'VERB'), ("'s", 'PRT')),
 (('firehose', '.'), ('firehose', 'NOUN')),
 (('Twitter', '.'), ('Twitter', 'NOUN')),
 (('online', '.'), ('online', 'ADJ')),
 (('interact', '.'), ('interact', 'NOUN')),
 (('messages', '.'), ('messages', 'NOUN')),
 (('tweets', '.'), ('tweets', 'NOUN')),
 (('domineering', '.'), ('domineering', 'ADJ')),
 (('personality', '.'), ('personality', 'NOUN')),
 (('2018', '.'), ('2018', 'NUM')),
 (('FIFA', '.'), ('FIFA', 'NOUN')),
 (('Cup', '.'), ('Cup'

### 2. EVALUATION ON MODIFIED VITERBI WITH TRANSITION PROBABILITY

In [39]:
# tagging the test sentences
start = time.time()
eval_tagged_seq_viterbi_proba = Viterbi_with_transition_proba(evaluation_sentences_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  51.748204946517944


In [40]:
sample_check_vit_proba = [i for i, j in zip(eval_tagged_seq_viterbi_proba, evaluation_sentences_run_base) if i == j] 
print("Number of words correctly tagged: ", len(sample_check_vit_proba))
print("Total Number of words: ", len(eval_tagged_seq_viterbi_proba))
accuracy = len(sample_check_vit_proba)/len(eval_tagged_seq_viterbi_proba)
print("Accuracy: ", accuracy)

Number of words correctly tagged:  157
Total Number of words:  181
Accuracy:  0.8674033149171271


**Incorrectly tagged words by viterbi with only transition proba for unknow words**

In [41]:
incorrect_tagged_cases_viterbi_proba = [j for i, j in enumerate(zip(eval_tagged_seq_viterbi_proba, evaluation_sentences_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_viterbi_proba

[(('Google', 'DET'), ('Google', 'NOUN')),
 (('smartphones', 'DET'), ('smartphones', 'NOUN')),
 (('2011', 'DET'), ('2011', 'NUM')),
 (('2013', 'DET'), ('2013', 'NUM')),
 (('2015', 'DET'), ('2015', 'NUM')),
 (('that', 'ADP'), ('that', 'DET')),
 (('Google', 'X'), ('Google', 'NOUN')),
 (('Twitter', 'VERB'), ('Twitter', 'NOUN')),
 (('firehose', 'VERB'), ('firehose', 'NOUN')),
 (('online', 'NOUN'), ('online', 'ADJ')),
 (('messages', 'DET'), ('messages', 'NOUN')),
 (('tweets', 'DET'), ('tweets', 'NOUN')),
 (('domineering', 'NOUN'), ('domineering', 'ADJ')),
 (('2018', 'NOUN'), ('2018', 'NUM')),
 (('21st', 'NOUN'), ('21st', 'NUM')),
 (('contested', 'NOUN'), ('contested', 'VERB')),
 (('11th', 'ADJ'), ('11th', 'NUM')),
 (('Show', 'NOUN'), ('Show', 'VERB')),
 (('like', 'ADP'), ('like', 'VERB')),
 (('Show', 'NOUN'), ('Show', 'VERB')),
 (('about', 'ADP'), ('about', 'ADV')),
 (('invited', 'NOUN'), ('invited', 'VERB')),
 (('experience', 'NOUN'), ('experience', 'VERB')),
 (('ICESAT-2', 'DET'), ('ICESAT

### 3. EVALUATION ON MODIFIED VITERBI WITH REGEX BACKOFF

In [42]:
# tagging the test sentences
start = time.time()
eval_tagged_seq_viterbi_with_regex = Viterbi_with_regex_backoff(evaluation_sentences_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  51.64611506462097


In [43]:
sample_check_vit_regx = [i for i, j in zip(eval_tagged_seq_viterbi_with_regex, evaluation_sentences_run_base) if i == j] 
print("Number of words correctly tagged: ", len(sample_check_vit_regx))
print("Total Number of words: ", len(eval_tagged_seq_viterbi_with_regex))
accuracy = len(sample_check_vit_regx)/len(eval_tagged_seq_viterbi_with_regex)
print("Accuracy: ", accuracy)

Number of words correctly tagged:  170
Total Number of words:  181
Accuracy:  0.9392265193370166


**Incorrectly tagged words by viterbi with regex based tagger for unknown words**

In [44]:
incorrect_tagged_cases_viterbi_regex = [j for i, j in enumerate(zip(eval_tagged_seq_viterbi_with_regex, evaluation_sentences_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases_viterbi_regex

[(('that', 'ADP'), ('that', 'DET')),
 (('firehose', 'VERB'), ('firehose', 'NOUN')),
 (('online', 'NOUN'), ('online', 'ADJ')),
 (('domineering', 'VERB'), ('domineering', 'ADJ')),
 (('11th', 'ADJ'), ('11th', 'NUM')),
 (('Show', 'NOUN'), ('Show', 'VERB')),
 (('like', 'ADP'), ('like', 'VERB')),
 (('Show', 'NOUN'), ('Show', 'VERB')),
 (('about', 'ADP'), ('about', 'ADV')),
 (('arriving', 'VERB'), ('arriving', 'NOUN')),
 (('experience', 'NOUN'), ('experience', 'VERB'))]


**FOLLOWING ARE THE ACCURACIES OF VITERBI ALGORITHMS ON EVALUATION SET:**
 

**1. VANILLA VITERBI: 76.24%**

**2. VITERBI WITH TRANSITION PROBABILITY FOR UNKNOWN WORDS: 86.74%**

**3. VITERBI WITH REGEX POS TAGGING: 93.92%**

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

### COMPARISON ON TEST SET:

Utility method which accepts 3 input.
      1. modified_viterbi_tags - tagger o/p of modified viterbi
      2. vanilla_viterbi_tags - output of vanilla viterbi
      3. base_tags - Orginal tag

Returns a list which contains those tags correctly tagged by modified viterbi compared to vanilla viterbi.

Output will be list of tuples, 
      1. on 0th position of tuple will be correct tag by modified viterbi
      2. on 1st position of tuple will be incorrect tag by vanilla viterbi
      

In [45]:
def check_corrected(modified_viterbi_tags, vanilla_viterbi_tags, base_tags):
    corrected = [(i, j) for i, j, k  in zip(modified_viterbi_tags, vanilla_viterbi_tags, base_tags) if (i == k) and (i != j)]
    return corrected

#### WORDS CORRECTLY CALSSIFIED BY VITERBI WITH TRANSITION PROBABILITY COMPARED TO VANILLA VITERBI ON TEST SET:

In [46]:
check_corrected(tagged_seq_mod_viterbi_1, tagged_seq_vanilla_viterbi, test_run_base)

[(('Overseas', 'NOUN'), ('Overseas', '.')),
 (('Unemployment', 'NOUN'), ('Unemployment', '.')),
 (('paycheck', 'NOUN'), ('paycheck', '.')),
 (('Funded', 'VERB'), ('Funded', '.')),
 (('protocols', 'NOUN'), ('protocols', '.')),
 (('existed', 'VERB'), ('existed', '.')),
 (('exists', 'VERB'), ('exists', '.')),
 (('acquirers', 'NOUN'), ('acquirers', '.')),
 (('noticed', 'VERB'), ('noticed', '.')),
 (('slate', 'NOUN'), ('slate', '.')),
 (('schoolchildren', 'NOUN'), ('schoolchildren', '.')),
 (('Catch-22', 'NOUN'), ('Catch-22', '.')),
 (('apologizing', 'VERB'), ('apologizing', '.')),
 (('indulging', 'VERB'), ('indulging', '.')),
 (('addiction', 'NOUN'), ('addiction', '.')),
 (('Mogavero', 'NOUN'), ('Mogavero', '.')),
 (('Piscataway', 'NOUN'), ('Piscataway', '.')),
 (('exploit', 'VERB'), ('exploit', '.')),
 (('reds', 'NOUN'), ('reds', '.')),
 (('Rhone', 'NOUN'), ('Rhone', '.')),
 (('copied', 'VERB'), ('copied', '.')),
 (('octogenarians', 'NOUN'), ('octogenarians', '.')),
 (('belfries', 'NOUN')

#### WORDS CORRECTLY CALSSIFIED BY VITERBI WITH REGEX TRAGGER BACKOFF COMPARED TO VANILLA VITERBI ON TEST SET:

In [47]:
check_corrected(tagged_seq_mod_viterbi_2, tagged_seq_vanilla_viterbi, test_run_base)

[(('Overseas', 'NOUN'), ('Overseas', '.')),
 (('pre-1917', 'ADJ'), ('pre-1917', '.')),
 (('Unemployment', 'NOUN'), ('Unemployment', '.')),
 (('paycheck', 'NOUN'), ('paycheck', '.')),
 (('reasonably', 'ADV'), ('reasonably', '.')),
 (('Tokio', 'NOUN'), ('Tokio', '.')),
 (('protocols', 'NOUN'), ('protocols', '.')),
 (('existed', 'VERB'), ('existed', '.')),
 (('20.5', 'NUM'), ('20.5', '.')),
 (('polarized', 'VERB'), ('polarized', '.')),
 (('acquirers', 'NOUN'), ('acquirers', '.')),
 (('noticed', 'VERB'), ('noticed', '.')),
 (('154.2', 'NUM'), ('154.2', '.')),
 (('touched', 'VERB'), ('touched', '.')),
 (('schoolchildren', 'NOUN'), ('schoolchildren', '.')),
 (('Catch-22', 'NOUN'), ('Catch-22', '.')),
 (('cross-border', 'ADJ'), ('cross-border', '.')),
 (('emigres', 'NOUN'), ('emigres', '.')),
 (('*T*-133', 'X'), ('*T*-133', '.')),
 (('apologizing', 'VERB'), ('apologizing', '.')),
 (('indulging', 'VERB'), ('indulging', '.')),
 (('1953', 'NUM'), ('1953', '.')),
 (('1955', 'NUM'), ('1955', '.'))

### COMPARISON ON EVALUATION SET:

#### WORDS CORRECTLY CALSSIFIED BY VITERBI WITH TRANSITION PROBABILITY COMPARED TO VANILLA VITERBI ON EVALUATION SET:

In [48]:
check_corrected(eval_tagged_seq_viterbi_proba, eval_tagged_seq_vanilla_viterbi, evaluation_sentences_run_base)

[(('Android', 'NOUN'), ('Android', '.')),
 (('Android', 'NOUN'), ('Android', '.')),
 (('OS', 'NOUN'), ('OS', '.')),
 (('worldwide', 'NOUN'), ('worldwide', '.')),
 (('Google', 'NOUN'), ('Google', '.')),
 (('Twitter', 'NOUN'), ('Twitter', '.')),
 (("'s", 'PRT'), ("'s", 'VERB')),
 (('Twitter', 'NOUN'), ('Twitter', '.')),
 (('interact', 'NOUN'), ('interact', '.')),
 (('personality', 'NOUN'), ('personality', '.')),
 (('FIFA', 'NOUN'), ('FIFA', '.')),
 (('Cup', 'NOUN'), ('Cup', '.')),
 (('FIFA', 'NOUN'), ('FIFA', '.')),
 (('Cup', 'NOUN'), ('Cup', '.')),
 (('tournament', 'NOUN'), ('tournament', '.')),
 (('Cup', 'NOUN'), ('Cup', '.')),
 (('trips', 'NOUN'), ('trips', '.')),
 (('arriving', 'NOUN'), ('arriving', '.')),
 (('NASA', 'NOUN'), ('NASA', '.')),
 (('Satellite', 'NOUN'), ('Satellite', '.'))]

#### WORDS CORRECTLY CALSSIFIED BY VITERBI WITH REGEX TRAGGER BACKOFF COMPARED TO VANILLA VITERBI ON EVALUATION SET:

In [49]:
check_corrected(eval_tagged_seq_viterbi_with_regex, eval_tagged_seq_vanilla_viterbi, evaluation_sentences_run_base)

[(('Android', 'NOUN'), ('Android', '.')),
 (('Google', 'NOUN'), ('Google', '.')),
 (('Android', 'NOUN'), ('Android', '.')),
 (('OS', 'NOUN'), ('OS', '.')),
 (('worldwide', 'NOUN'), ('worldwide', '.')),
 (('smartphones', 'NOUN'), ('smartphones', '.')),
 (('2011', 'NUM'), ('2011', '.')),
 (('2013', 'NUM'), ('2013', '.')),
 (('Google', 'NOUN'), ('Google', '.')),
 (('Twitter', 'NOUN'), ('Twitter', '.')),
 (('2015', 'NUM'), ('2015', '.')),
 (('Google', 'NOUN'), ('Google', '.')),
 (('Twitter', 'NOUN'), ('Twitter', '.')),
 (("'s", 'PRT'), ("'s", 'VERB')),
 (('Twitter', 'NOUN'), ('Twitter', '.')),
 (('interact', 'NOUN'), ('interact', '.')),
 (('messages', 'NOUN'), ('messages', '.')),
 (('tweets', 'NOUN'), ('tweets', '.')),
 (('personality', 'NOUN'), ('personality', '.')),
 (('2018', 'NUM'), ('2018', '.')),
 (('FIFA', 'NOUN'), ('FIFA', '.')),
 (('Cup', 'NOUN'), ('Cup', '.')),
 (('21st', 'NUM'), ('21st', '.')),
 (('FIFA', 'NOUN'), ('FIFA', '.')),
 (('Cup', 'NOUN'), ('Cup', '.')),
 (('tournament'

## CONCLUSION:

**FOLLOWING ARE THE ACCURACIES OF VITERBI ALGORITHMS ON TEST SET (CREATED FROM TRAIN TEST SPLIT) :**
    
1. VANILLA VITERBI: 91.39%
2. VITERBI WITH TRANSITION PROBABILITY FOR UNKNOWN WORDS: 93.89%
3. VITERBI WITH REGEX POS TAGGING: 96.26%

**EVALUATION ON TEST SENTENCES AS GIVEN PART OF "Test_sentences.txt"**

1. VANILLA VITERBI: 76.24%
2. VITERBI WITH TRANSITION PROBABILITY FOR UNKNOWN WORDS: 86.74%
3. VITERBI WITH REGEX POS TAGGING: 93.92%

As we can see from above results on test data as well as on evaluating on "Test_sentences.txt" the VITERBI with Regex Rule Based modification resulted in higher accuracy compared to vanilla viterbi algorithm.